In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import polars as pl
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [17]:
COORDINATES = ["chrom", "pos", "ref", "alt"]


dataset = "mendelian_matched_9"
#dataset = "gwas_matched_9"

subset = "non_coding"
#model = "CADD"
model = "MAF"

base_dir = f"../../results/dataset/{dataset}"
df = pd.read_csv(f"{base_dir}/unsupervised_metrics/{subset}/{model}.csv")
df

FileNotFoundError: [Errno 2] No such file or directory: '../../results/dataset/mendelian_matched_9/unsupervised_metrics/non_coding/MAF.csv'

In [4]:
df.head(50)

,AUPRC,feature
0,0.679601,RawScore
1,0.660773,ZooVerPhyloP
2,0.627462,GerpS
3,0.613003,verPhyloP
4,0.565781,verPhCons
5,0.560215,mamPhyloP
6,0.533279,mamPhCons
7,0.495663,Freq100bp
8,0.492913,priPhCons
9,0.478686,ZooPriPhyloP


In [35]:
V = pd.read_parquet(f"{base_dir}/test.parquet")
V.shape, V.drop_duplicates(subset=COORDINATES).shape

((11770, 10), (11769, 10))

In [19]:
V[V[COORDINATES].duplicated(keep=False)]

,chrom,pos,ref,alt,consequence,source,OMIM,label,tss_dist,match_group
4250,8,19939160,T,G,PLS,OMIM,MIM 238600,True,92,PLS_57
4251,8,19939160,T,G,PLS,gnomAD,None,False,92,PLS_57


In [36]:
X = pd.read_parquet(f"{base_dir}/features/{model}.parquet")
features = X.columns
V = pd.concat([V, X], axis=1)
V

,chrom,pos,ref,alt,consequence,source,OMIM,label,tss_dist,match_group,score
0,1,976215,A,G,missense_variant,gnomAD,None,False,2702,missense_variant_49,0.285295
1,1,978953,C,G,missense_variant,gnomAD,None,False,3139,missense_variant_30,0.468749
2,1,1243545,G,A,missense_variant,gnomAD,None,False,3176,missense_variant_30,0.107950
3,1,1264812,C,G,missense_variant,gnomAD,None,False,8658,missense_variant_23,0.081697
4,1,1425822,C,G,PLS,gnomAD,None,False,48,PLS_4,0.096771
...,...,...,...,...,...,...,...,...,...,...,...
11765,X,156021686,C,T,missense_variant,gnomAD,None,False,724,missense_variant_776,0.098962
11766,X,156022062,G,A,missense_variant,gnomAD,None,False,1100,missense_variant_784,0.138343
11767,X,156022092,C,T,missense_variant,gnomAD,None,False,1130,missense_variant_790,0.063535
11768,X,156023118,G,T,missense_variant,gnomAD,None,False,2156,missense_variant_778,0.077227


In [38]:
V2 = V[V.consequence == "missense_variant"]
V2 = V2.query("label")
V2.score = V2.score.fillna(0)
V2

,chrom,pos,ref,alt,consequence,source,OMIM,label,tss_dist,match_group,score
63,1,11114338,A,G,missense_variant,ClinVar,None,True,14468,missense_variant_0,0.000000
64,1,11124516,G,A,missense_variant,ClinVar,None,True,24646,missense_variant_1,0.000000
65,1,11124516,G,T,missense_variant,ClinVar,None,True,24646,missense_variant_2,0.000000
66,1,11128107,G,C,missense_variant,ClinVar,None,True,28237,missense_variant_3,0.000000
67,1,11128107,G,T,missense_variant,ClinVar,None,True,28237,missense_variant_4,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
11743,X,154957060,C,T,missense_variant,ClinVar,None,True,65662,missense_variant_769,0.000000
11744,X,154966041,G,A,missense_variant,ClinVar,None,True,56681,missense_variant_782,0.000000
11745,X,154966603,T,C,missense_variant,ClinVar,None,True,56119,missense_variant_787,0.000063
11746,X,154969361,G,C,missense_variant,ClinVar,None,True,53361,missense_variant_795,0.000000


In [39]:
V2[V2.score > 0.1 / 100]

,chrom,pos,ref,alt,consequence,source,OMIM,label,tss_dist,match_group,score
705,1,216247118,C,A,missense_variant,ClinVar,None,True,176277,missense_variant_55,0.001308
6984,13,20189473,C,T,missense_variant,ClinVar,None,True,306,missense_variant_218,0.004002
6985,13,20189481,A,G,missense_variant,ClinVar,None,True,298,missense_variant_219,0.009420
8353,17,7222272,T,C,missense_variant,ClinVar,None,True,925,missense_variant_341,0.001092


In [21]:
V = V[V.consequence != "missense_variant"]
V

,chrom,pos,ref,alt,consequence,source,OMIM,label,tss_dist,match_group,score
4,1,1425822,C,G,PLS,gnomAD,None,False,48,PLS_4,0.096771
6,1,1615869,C,T,PLS,gnomAD,None,False,35,PLS_0,0.374647
7,1,1659060,G,A,PLS,gnomAD,None,False,47,PLS_4,0.293482
14,1,2050958,T,C,5_prime_UTR_variant,gnomAD,None,False,149,5_prime_UTR_variant_7,0.067459
15,1,2074688,G,A,5_prime_UTR_variant,gnomAD,None,False,0,5_prime_UTR_variant_2,0.227352
...,...,...,...,...,...,...,...,...,...,...,...
11758,X,155613005,C,T,PLS,gnomAD,None,False,52,PLS_62,0.368939
11759,X,155719093,C,A,5_prime_UTR_variant,gnomAD,None,False,4,5_prime_UTR_variant_109,0.100729
11760,X,155881342,A,C,PLS,gnomAD,None,False,2,PLS_67,0.095063
11761,X,155881414,C,T,5_prime_UTR_variant,gnomAD,None,False,35,5_prime_UTR_variant_115,0.193242


In [22]:
V.groupby("label").score.mean()

label
False    0.219027
True     0.002878
Name: score, dtype: float64

In [24]:
V.query("label").score.describe()

count    145.000000
mean       0.002878
std        0.012009
min        0.000000
25%        0.000013
50%        0.000033
75%        0.000368
max        0.120373
Name: score, dtype: float64

In [25]:
V.query("not label").score.describe()

count    3366.000000
mean        0.219027
std         0.133394
min         0.050016
25%         0.097532
50%         0.189547
75%         0.327265
max         0.499625
Name: score, dtype: float64

In [26]:
V = V.query("label")
V

,chrom,pos,ref,alt,consequence,source,OMIM,label,tss_dist,match_group,score
47,1,7961859,C,G,PLS,OMIM,MIM 606324,True,34,PLS_0,0.000106
55,1,9943502,A,T,5_prime_UTR_variant,OMIM,MIM 608553,True,26,5_prime_UTR_variant_0,NaN
56,1,9943503,C,T,5_prime_UTR_variant,OMIM,MIM 608553,True,27,5_prime_UTR_variant_1,NaN
60,1,11023351,G,A,3_prime_UTR_variant,OMIM,MIM 612069,True,1206,3_prime_UTR_variant_0,NaN
139,1,21509427,C,T,5_prime_UTR_variant,OMIM,MIM 241500,True,0,5_prime_UTR_variant_2,NaN
...,...,...,...,...,...,...,...,...,...,...,...
11749,X,155022770,A,G,PLS,OMIM,MIM 306700,True,46,PLS_69,NaN
11750,X,155022771,G,A,PLS,OMIM,MIM 306700,True,47,PLS_70,NaN
11751,X,155022773,A,T,PLS,OMIM,MIM 306700,True,49,PLS_71,NaN
11752,X,155022807,T,C,PLS,OMIM,MIM 306700,True,83,PLS_72,NaN


In [27]:
V.score = V.score.fillna(0)
V

,chrom,pos,ref,alt,consequence,source,OMIM,label,tss_dist,match_group,score
47,1,7961859,C,G,PLS,OMIM,MIM 606324,True,34,PLS_0,0.000106
55,1,9943502,A,T,5_prime_UTR_variant,OMIM,MIM 608553,True,26,5_prime_UTR_variant_0,0.000000
56,1,9943503,C,T,5_prime_UTR_variant,OMIM,MIM 608553,True,27,5_prime_UTR_variant_1,0.000000
60,1,11023351,G,A,3_prime_UTR_variant,OMIM,MIM 612069,True,1206,3_prime_UTR_variant_0,0.000000
139,1,21509427,C,T,5_prime_UTR_variant,OMIM,MIM 241500,True,0,5_prime_UTR_variant_2,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
11749,X,155022770,A,G,PLS,OMIM,MIM 306700,True,46,PLS_69,0.000000
11750,X,155022771,G,A,PLS,OMIM,MIM 306700,True,47,PLS_70,0.000000
11751,X,155022773,A,T,PLS,OMIM,MIM 306700,True,49,PLS_71,0.000000
11752,X,155022807,T,C,PLS,OMIM,MIM 306700,True,83,PLS_72,0.000000


In [31]:
V[V.score > 0.1 / 100].shape

(27, 11)

In [32]:
V[V.score > 0.1 / 100]

,chrom,pos,ref,alt,consequence,source,OMIM,label,tss_dist,match_group,score
1061,2,69326167,G,T,3_prime_UTR_variant,OMIM,MIM 610542,True,60962,3_prime_UTR_variant_26,0.001430
1081,2,71686599,T,A,3_prime_UTR_variant,OMIM,MIM 253601,True,109699,3_prime_UTR_variant_27,0.006374
1518,2,202376806,G,A,5_prime_UTR_variant,OMIM,MIM 178600,True,305,5_prime_UTR_variant_83,0.006093
2086,3,49171662,C,T,5_prime_UTR_variant,OMIM,MIM 236000,True,63,5_prime_UTR_variant_90,0.004716
2857,5,112737311,A,T,pELS,OMIM,MIM 175100,True,573,pELS_0,0.040496
2895,5,177409584,C,G,PLS,OMIM,MIM 234000,True,19,PLS_51,0.002846
2896,5,177409589,G,A,PLS,OMIM,MIM 234000,True,24,PLS_52,0.002175
2931,6,118548260,A,C,PLS,OMIM,MIM 609909,True,35,PLS_54,0.040333
3783,7,107660756,T,C,non_coding_transcript_exon_variant,OMIM,MIM 274600,True,71,non_coding_transcript_exon_variant_49,0.002010
3802,7,117479283,T,G,5_prime_UTR_variant,OMIM,MIM 219700,True,704,5_prime_UTR_variant_95,0.003729


In [8]:
subset_df = pd.read_parquet(f"{base_dir}/subset/{subset}.parquet")
subset_df

,chrom,pos,ref,alt
0,1,1425822,C,G
1,1,1615869,C,T
2,1,1659060,G,A
3,1,2050958,T,C
4,1,2074688,G,A
...,...,...,...,...
3705,X,155613005,C,T
3706,X,155719093,C,A
3707,X,155881342,A,C
3708,X,155881414,C,T


In [9]:
V = V.merge(subset_df, how="inner", on=["chrom", "pos", "ref", "alt"])
V

,chrom,pos,ref,alt,consequence,source,OMIM,label,tss_dist,match_group,...,RegSeq5,RegSeq6,RegSeq7,Aparent2,ZooPriPhyloP,ZooVerPhyloP,ZooRoCC,Roulette-MR,Roulette-AR,RawScore
0,1,1425822,C,G,PLS,gnomAD,None,False,48,PLS_4,...,0.009538,0.009359,-0.008527,-0.0013,0.003,0.172,0.0,0.030,0.030,0.641057
1,1,1615869,C,T,PLS,gnomAD,None,False,35,PLS_0,...,0.038628,0.044415,-0.034095,-0.0013,0.015,-2.128,0.0,0.274,0.288,0.168522
2,1,1659060,G,A,PLS,gnomAD,None,False,47,PLS_4,...,0.001092,0.005135,-0.000776,-0.0013,0.005,-1.945,0.0,0.163,0.163,0.040859
3,1,2050958,T,C,5_prime_UTR_variant,gnomAD,None,False,149,5_prime_UTR_variant_7,...,-0.019481,-0.005475,-0.001035,-0.0013,0.000,0.952,0.0,0.371,0.329,0.382432
4,1,2074688,G,A,5_prime_UTR_variant,gnomAD,None,False,0,5_prime_UTR_variant_2,...,-0.000654,-0.001675,-0.003910,-0.0013,0.000,-1.770,0.0,1.833,0.105,-0.528766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3707,X,155613005,C,T,PLS,gnomAD,None,False,52,PLS_62,...,-0.008340,-0.011424,0.017324,-0.0013,0.000,-1.642,0.0,0.094,0.105,-0.046796
3708,X,155719093,C,A,5_prime_UTR_variant,gnomAD,None,False,4,5_prime_UTR_variant_109,...,-0.055994,-0.049813,0.034946,-0.0013,0.003,0.316,0.0,0.094,0.105,0.301978
3709,X,155881342,A,C,PLS,gnomAD,None,False,2,PLS_67,...,0.030061,0.025888,-0.031599,-0.0013,0.032,-0.365,0.0,0.094,0.105,0.325166
3710,X,155881414,C,T,5_prime_UTR_variant,gnomAD,None,False,35,5_prime_UTR_variant_115,...,-0.022957,-0.023271,0.028425,-0.0013,0.001,-0.290,0.0,0.094,0.105,0.412759


In [11]:
V[["chrom", "pos", "ref", "alt"]].drop_duplicates().shape

(3709, 4)

In [ ]:
# does this number make sense?